In [1]:
from collections import defaultdict
import json
import tqdm

import numpy as np
from openff.toolkit import Molecule, ForceField

In [2]:
QCFRACTAL_URL = "https://api.qcarchive.molssi.org:443/"

## Filter existing records

In [3]:
def filter_dataset(file, searches) -> dict[int, str]:
    """
    Quickly filter for records of interest.
    """
    output = {}
    seen_smiles_nonmatches = set()
    seen_smiles_matches = set()

    ff = ForceField("openff-2.2.1.offxml")

    with open(file, "r") as f:
        
        entries = json.load(f)["entries"][QCFRACTAL_URL]
        for entry in tqdm.tqdm(entries):
            smi = entry["cmiles"]
            if smi in seen_smiles_matches:
                output[int(entry["record_id"])] = smi
                continue
            elif smi in seen_smiles_nonmatches:
                continue
                
            mol = Molecule.from_smiles(smi, allow_undefined_stereo=True)
            torsions = [
                param.id for param in ff.label_molecules(
                    mol.to_topology()
                )[0]["ProperTorsions"].values()
            ]
            if any(torsion in torsions for torsion in searches):
                output[int(entry["record_id"])] = smi
                seen_smiles_matches.add(smi)
            else:
                seen_smiles_nonmatches.add(smi)
    return output

In [4]:
# quickly filter through existing data
searches = ["t17", "t48a", "t19", "t18", "t105"]
opt_record_ids = filter_dataset("sage-2.2.1/optimization-training-set.json", searches)
td_record_ids = filter_dataset("sage-2.2.1/torsion-training-set.json", searches)

/var/folders/bf/2ytrzlmd6jbbgqs37bl9lld40000gn/T/ipykernel_25751/530512539.py:22: AtomMappingWarning: Warning! Fully mapped SMILES pattern passed to `from_smiles`. The atom map is stored as a property in `Molecule._properties`, but these indices are NOT used to determine atom ordering. To use these indices for atom ordering, use `Molecule.from_mapped_smiles`.
  mol = Molecule.from_smiles(smi, allow_undefined_stereo=True)
100%|██████████████████████████████████████| 1290/1290 [00:11<00:00, 110.96it/s]


In [5]:
print(len(opt_record_ids))
print(len(td_record_ids))

3939
693


In [6]:
from openff.qcsubmit.results import (
    BasicResultCollection,
    OptimizationResultCollection,
    TorsionDriveResultCollection,
)

import qcportal as ptl

In [7]:
client = ptl.PortalClient(QCFRACTAL_URL, cache_dir=".")

In [8]:
# pull energies and forces for opts, energies for torsions

In [9]:
optimization_records_original = client.get_optimizations(
    list(opt_record_ids),
    include=["initial_molecule"]
)

In [10]:
optimization_records_original[0].final_molecule.identifiers

Identifiers(molecule_hash='7b42c896d68faf7b09b4d93149a769e47b5476bd', molecular_formula='C13H13NO2', smiles=None, inchi=None, inchikey=None, canonical_explicit_hydrogen_smiles=None, canonical_isomeric_explicit_hydrogen_mapped_smiles=None, canonical_isomeric_explicit_hydrogen_smiles=None, canonical_isomeric_smiles=None, canonical_smiles=None, pubchem_cid=None, pubchem_sid=None, pubchem_conformerid=None)

In [11]:
# this might be the lightest way to simply fetch the last frame
# alternatively could do `includes=['trajectory']`
# but I think this fetches the entire trajectory one-by-one

singlepoints_original_with_smiles = [
    (opt_record_ids[opt.id], opt.trajectory_element(-1))
    for opt in tqdm.tqdm(
        optimization_records_original,
        desc="Fetching last frame",
    )
]

Fetching last frame: 100%|██████████████████| 3939/3939 [43:08<00:00,  1.52it/s]


In [12]:
singlepoints_original_with_smiles[0][0]

'[H:17][c:1]1[c:2]([c:5]([c:9]([c:8]([c:4]1[H:20])[c:10]2[c:6]([c:3]([c:7]([c:11](=[O:15])[n:14]2[C:12]([H:24])([H:25])[H:26])[H:23])[H:19])[H:22])[O:16][C:13]([H:27])([H:28])[H:29])[H:21])[H:18]'

In [13]:
torsiondrive_records_original = client.get_records(
    list(td_record_ids),
    include=["minimum_optimizations"]
)

In [14]:
torsiondrive_records_original[0]

TorsiondriveRecord(id=18535805, record_type='torsiondrive', is_service=True, properties={}, extras={}, status=<RecordStatusEnum.complete: 'complete'>, manager_name=None, created_on=datetime.datetime(2020, 3, 24, 17, 35, 56, 324581, tzinfo=datetime.timezone.utc), modified_on=datetime.datetime(2020, 3, 24, 17, 35, 56, 324579, tzinfo=datetime.timezone.utc), owner_user=None, owner_group=None, compute_history_=None, task_=None, service_=None, comments_=None, native_files_=None, specification=TorsiondriveSpecification(program='torsiondrive', optimization_specification=OptimizationSpecification(program='geometric', qc_specification=QCSpecification(program='psi4', driver=<SinglepointDriver.deferred: 'deferred'>, method='b3lyp-d3bj', basis='dzvp', keywords={'maxiter': 200, 'scf_properties': ['dipole', 'quadrupole', 'wiberg_lowdin_indices', 'mayer_indices']}, protocols=AtomicResultProtocols(wavefunction=<WavefunctionProtocolEnum.none: 'none'>, stdout=True, error_correction=ErrorCorrectionProtoco

In [15]:
singlepoints_torsiondrive_original = []
for record in tqdm.tqdm(torsiondrive_records_original, desc="Pulling last frames"):
    smi = td_record_ids[record.id]

    # first check this is rotating around the correct torsions
    mol = Molecule.from_mapped_smiles(smi, allow_undefined_stereo=True)
    ff = ForceField("openff-2.2.1.offxml")
    torsions = ff.label_molecules(
        mol.to_topology()
    )[0]["ProperTorsions"]
    
    dihedrals = record.specification.keywords.dihedrals
    scanned_torsions = []
    for dih in dihedrals:
        if dih in torsions:
            scanned_torsions.append(torsions[dih].id)
        elif dih[::-1] in torsions:
            scanned_torsions.append(torsions[dih[::-1]].id)

    if not any(tors in scanned_torsions for tors in searches):
        continue

    # now pull the constrained optimization
    for opt in record.minimum_optimizations.values():
        singlepoints_torsiondrive_original.append(
            (smi, opt.trajectory_element(-1))
        )
        
print(len(singlepoints_torsiondrive_original))

Pulling last frames: 100%|████████████████████| 693/693 [18:39<00:00,  1.62s/it]

1560


## Pull down additional complete datasets

In [16]:
# Pull down the records from additional datasets
additional_torsiondrive_collection = TorsionDriveResultCollection.from_server(
    client=client,
    datasets="OpenFF Cresset Additional Coverage TorsionDrives v4.0",
    spec_name="default",
)
additional_optimization_collection = OptimizationResultCollection.from_server(
    client=client,
    datasets="OpenFF Cresset Additional Coverage Optimizations v4.0",
    spec_name="default",
)

In [17]:
additional_td_records_and_molecules = list(
    additional_torsiondrive_collection.to_records()
)
additional_opt_records_and_molecules = list(
    additional_optimization_collection.to_records()
)


In [18]:
additional_opt_records_and_molecules[0][0].trajectory_element(-1).molecule.identifiers

Identifiers(molecule_hash='6cbc29d00b72458a7ec1dd141c14a76d35d4fbe4', molecular_formula='C8H11N3', smiles=None, inchi=None, inchikey=None, canonical_explicit_hydrogen_smiles=None, canonical_isomeric_explicit_hydrogen_mapped_smiles='[H:14][C:3]1=[C:2]([N:11]([N:10]=[C:4]1[C:5]23[C:6]([C:7]([C:8]2([H:18])[H:19])([C:9]3([H:20])[H:21])[H:17])([H:15])[H:16])[H:22])[N:1]([H:12])[H:13]', canonical_isomeric_explicit_hydrogen_smiles=None, canonical_isomeric_smiles=None, canonical_smiles=None, pubchem_cid=None, pubchem_sid=None, pubchem_conformerid=None)

In [19]:
singlepoints_additional = []
for opt, _ in tqdm.tqdm(additional_opt_records_and_molecules):
    last = opt.trajectory_element(-1)
    singlepoints_additional.append(
        (last.molecule.identifiers.canonical_isomeric_explicit_hydrogen_mapped_smiles, last)
    )

100%|█████████████████████████████████████████| 393/393 [05:57<00:00,  1.10it/s]


In [20]:
singlepoints_torsiondrive_additional = []
for record, _ in tqdm.tqdm(additional_td_records_and_molecules, desc="Pulling last frames"):
    for opt in record.minimum_optimizations.values():
        last = opt.trajectory_element(-1)
        singlepoints_torsiondrive_additional.append(
            (last.molecule.identifiers.canonical_isomeric_explicit_hydrogen_mapped_smiles, last)
        )

Pulling last frames: 100%|██████████████████████| 82/82 [32:08<00:00, 23.52s/it]


## Convert to JSON

In [21]:
from openff.units import unit

bohr_to_angstrom = (1 * unit.bohr).m_as(unit.angstrom)
hartree_to_kcal = (1 * unit.hartree * unit.avogadro_constant).m_as(
    unit.kilocalories_per_mole
)

def convert_singlepoints(singlepoints, include_gradient: bool = True):
    """
    Convert data to smee-friendly format.
    """
    data_by_smiles = defaultdict(list)

    for mapped_smiles, record in tqdm.tqdm(singlepoints, desc="Converting"): 
        mol = record.molecule
        coords = np.array(mol.geometry * bohr_to_angstrom).flatten().tolist()
        energy = record.properties["return_energy"] * hartree_to_kcal
        entry = {
            "coords": coords,
            "energy": energy,
        }
        if include_gradient:
            try:
                gradient = np.array(record.properties["scf total gradient"]).reshape((-1, 3))
            except KeyError:
                # throw out any opts without gradients for now
                continue
            forces = ((-gradient) * hartree_to_kcal / bohr_to_angstrom)
            entry["forces"] = np.array(forces).flatten().tolist()
            
        data_by_smiles[mapped_smiles].append(entry)
    return data_by_smiles

In [23]:
# singlepoints_original[0].molecule.identifiers

In [25]:
# original_optimizations = convert_singlepoints(singlepoints_original)
# len(original_optimizations)

In [26]:
# convert optimizations

original_optimizations = convert_singlepoints(singlepoints_original_with_smiles)
with open("original-optimizations.json", "w") as f:
    json.dump(original_optimizations, f)
    
additional_optimizations = convert_singlepoints(singlepoints_additional)
with open("additional-optimizations.json", "w") as f:
    json.dump(additional_optimizations, f)

original_torsiondrives = convert_singlepoints(
    singlepoints_torsiondrive_original,
    include_gradient=False
)
with open("original-torsiondrives.json", "w") as f:
    json.dump(original_torsiondrives, f)
    
additional_torsiondrives = convert_singlepoints(
    singlepoints_torsiondrive_additional,
    include_gradient=False
)
with open("additional-torsiondrives.json", "w") as f:
    json.dump(additional_torsiondrives, f)


Converting: 100%|████████████████████████| 1968/1968 [00:00<00:00, 34056.28it/s]


NameError: name 'collections' is not defined

In [27]:
import collections

In [28]:
combined_optimizations = collections.defaultdict(list)
combined_torsiondrives = collections.defaultdict(list)

for dataset in [original_optimizations, additional_optimizations]:
    for k, v in dataset.items():
        combined_optimizations[k].extend(v)

for dataset in [original_torsiondrives, additional_torsiondrives]:
    for k, v in dataset.items():
        combined_torsiondrives[k].extend(v)

with open("combined-optimizations.json", "w") as f:
    json.dump(combined_optimizations, f)

with open("combined-torsiondrives.json", "w") as f:
    json.dump(combined_torsiondrives, f)
